In [1]:
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment
from pyflink.table.expressions import col, lit

In [2]:
# set the streaming environment
s_env = StreamExecutionEnvironment.get_execution_environment()

# add the kafka connector 
s_env.add_jars("file:///C:/Users/Carter%20Dakota/portfolio/downloads/flink-sql-connector-kafka-3.2.0-1.19.jar")

In [3]:
# set the table environment for configuring in SQL
t_env = StreamTableEnvironment.create(s_env)

In [10]:
# set the source
t_env.execute_sql("""
    CREATE TABLE kafka_source (
        `song` string ,
        `artist` string ,
        `duration` double ,
        `ts` bigint ,
        `sessionId` INTEGER ,
        `auth` string ,
        `level` string ,
        `itemInSession` INTEGER ,
        `city` string ,
        `zip` string ,
        `state` string ,
        `userAgent` string ,
        `lon` double ,
        `lat` double ,
        `userId` integer ,
        `lastName` string ,
        `firstName` string ,
        `gender` string ,
        `registration` BIGINT
    ) WITH (
        'connector' = 'kafka',
        'topic' = 'listen_events',
        'properties.bootstrap.servers' = '3.96.62.156:9092',
        'properties.group.id' = 'flink-consumer-group-1', 
        'scan.startup.mode' = 'earliest-offset',
        'format' = 'json'
    )
""")

In [9]:
t_env.execute_sql("""
    CREATE TABLE listen_events_s3 (
        `song` string ,
        `artist` string ,
        `duration` double ,
        `ts` bigint ,
        `sessionId` INTEGER ,
        `auth` string ,
        `level` string ,
        `itemInSession` INTEGER ,
        `city` string ,
        `zip` string ,
        `state` string ,
        `userAgent` string ,
        `lon` double ,
        `lat` double ,
        `userId` integer ,
        `lastName` string ,
        `firstName` string ,
        `gender` string ,
        `registration` BIGINT
        ) WITH (
            'connector' = 'filesystem',
            'path' = 's3://streaming-demo-project/outputs',
            'format' = 'parquet',
            'sink.rolling-policy.file-size' = '128MB',
            'sink.rolling-policy.rollover-interval' = '15 min',
            'sink.rolling-policy.check-interval' = '1 min',
            'partition.time-extractor.timestamp-pattern'='$year-$month-$day 00:00:00',
            'sink.partition-commit.delay'='1 h',
            'sink.partition-commit.policy.kind'='success-file'
        )
""")

In [11]:
kafka_source = t_env.from_path('kafka_source')

In [12]:
t_env.execute_sql("""
        INSERT INTO sink_print
        SELECT *
        FROM kafka_source; 
""")

In [18]:
result = t_env.from_path('print')